In [1]:
# ALWAYS RUN THIS CELL
# widen jupyter notebook window
from IPython.display import display, HTML
display(HTML("<style>.container {width:95% !important; }</style>"))

This requires having `bnpm`. Get it with `pip install bnpm` or here: https://github.com/RichieHakim/basic_neural_processing_modules

In [2]:
%load_ext autoreload
%autoreload 2
from bnpm import server

In [3]:
import os
from pathlib import Path
import functools
import tempfile
import getpass
import gc

In [57]:
import numpy as np
import natsort

------------
# ==== Start ====

## Prepare username + password

In [4]:
remote_host_transfer = "transfer.rc.hms.harvard.edu"
remote_host_compute = "o2.hms.harvard.edu"
username = input('Username: ')

use_localSshKey = False

pw = server.pw_encode(getpass.getpass(prompt='Password: ')) if use_localSshKey==False else None

path_sshKey = '/home/rich/.ssh/id_rsa' if use_localSshKey else None

Username: rh183
Password: ········


## Prepare `sftp` and `ssh` objects

In [5]:
## initialize ssh_transfer
ssh_t = server.ssh_interface(
    nbytes_toReceive=20000,
    recv_timeout=1,
    verbose=True,
)

In [6]:
ssh_t.o2_connect(
    hostname=remote_host_transfer,
    username=username,
    password=server.pw_decode(pw),
    key_filename=path_sshKey,
    look_for_keys=False,
    passcode_method=1,
    verbose=0,
    skip_passcode=False,    
)

In [7]:
## initialize ssh_compute
ssh_c = server.ssh_interface(
    nbytes_toReceive=20000,
    recv_timeout=1,
    verbose=True,
)
ssh_c.o2_connect(
    hostname=remote_host_compute,
    username=username,
    password=server.pw_decode(pw),
    key_filename=path_sshKey,
    look_for_keys=False,
    passcode_method=1,
    verbose=0,
    skip_passcode=False,    
)

In [8]:
sftp = server.sftp_interface(ssh_client=ssh_t.client)

In [9]:
prompt_snip = f'[{username}'

In [10]:
send_expect = functools.partial(ssh_c.send_expect, str_success=prompt_snip)

# Batch run

In [16]:
paths_found = sftp.search_recursive(
    path='/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701',
    search_pattern_re='TCA.h5',
    max_depth=5,
    find_folders=False,
    find_files=True,
)

/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230531/jobNum_0/analysis_files/TCA.h5
/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230523/jobNum_0/analysis_files/TCA.h5
/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230517/jobNum_0/analysis_files/TCA.h5
/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230515/jobNum_0/analysis_files/TCA.h5
/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230430/jobNum_0/analysis_files/TCA.h5
/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230528/jobNum_0/analysis_files/TCA.h5
/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230601/jobNum_0/analysis_files/TCA.h5
/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230512/jobNum_0/analysis_files/TCA.h5
/n/data1/hms/neurobio/sabatini/r

In [69]:
paths_found = sftp.search_recursive(
    path='/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701',
    search_pattern_re='tca_refit.h5',
    max_depth=5,
    find_folders=False,
    find_files=True,
)

/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230531/aligned_TCA/jobNum_0/tca_refit.h5
/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230517/aligned_TCA/jobNum_0/tca_refit.h5
/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230515/aligned_TCA/jobNum_0/tca_refit.h5
/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230430/aligned_TCA/jobNum_0/tca_refit.h5
/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230528/aligned_TCA/jobNum_0/tca_refit.h5
/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230601/aligned_TCA/jobNum_0/tca_refit.h5
/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230511/aligned_TCA/jobNum_0/tca_refit.h5
/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230514/aligned_TCA/jobNum_0/tca_refit.h5
/n/data1

In [58]:
natsort.natsorted(paths_found)

['/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230430/aligned_TCA/jobNum_0/tca_refit.h5',
 '/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230501/aligned_TCA/jobNum_0/tca_refit.h5',
 '/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230502/aligned_TCA/jobNum_0/tca_refit.h5',
 '/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230504/aligned_TCA/jobNum_0/tca_refit.h5',
 '/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230507/aligned_TCA/jobNum_0/tca_refit.h5',
 '/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230508/aligned_TCA/jobNum_0/tca_refit.h5',
 '/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230509/aligned_TCA/jobNum_0/tca_refit.h5',
 '/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230511/aligned_TCA/

In [20]:
dates_toUse = natsort.natsorted([Path(p).parts[-4] for p in paths_found])
dates_toUse

['20230430',
 '20230501',
 '20230502',
 '20230503',
 '20230504',
 '20230505',
 '20230506',
 '20230507',
 '20230508',
 '20230509',
 '20230510',
 '20230511',
 '20230512',
 '20230514',
 '20230515',
 '20230516',
 '20230517',
 '20230518',
 '20230519',
 '20230520',
 '20230521',
 '20230522',
 '20230523',
 '20230524',
 '20230525',
 '20230526',
 '20230527',
 '20230528',
 '20230529',
 '20230530',
 '20230531',
 '20230601']

In [59]:
dates_toUse = [
#     '20230430',
#     '20230501',
#     '20230502',
    '20230503',
#     '20230504',
    '20230505',
    '20230506',
#     '20230507',
#     '20230508',
#     '20230509',
    '20230510',
#     '20230511',
    '20230512',
#     '20230514',
#     '20230515',
    '20230516',
#     '20230517',
#     '20230518',
#     '20230519',
#     '20230520',
#     '20230521',
#     '20230522',
    '20230523',
    '20230524',
    '20230525',
    '20230526',
#     '20230527',
#     '20230528',
#     '20230529',
    '20230530',
#     '20230531',
#     '20230601',
]

## Start a small interactive node

In [66]:
send_expect(f'srun --pty -p interactive -t 0-00:30:00 --mem=4G /bin/bash');

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0
Unpacking objects: 100% (4/4), done.
From https://github.com/RichieHakim/NBAP
   f197eeb..e767517  main       -> origin/main
Updating f197eeb..e767517
Fast-forward
 behavioral_drift_analysis/1_make_consistent_fr_factors.py | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)
(base) [rh183@login03 NBAP]$ 


## Pull/update repo

In [65]:
send_expect('cd /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP')
send_expect('git pull');

rtual_envs/FRum_     mouse_0322N_0530     /n/data1/hms/neurobio/sabatini/rich/vi 
dispatcher environment: /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR
Copying /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/1_make_consistent_fr_factors.py to /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230530/aligned_TCA/1_make_consistent_fr_factors.py
Submitting job: jobNum_ 0
Submitted batch job 12686423
AP]$ srun: Force Terminated job 12686091tual_envs/FR) [rh183@compute-e-16-233 NBA
slurmstepd: error: *** STEP 12686091.0 ON compute-e-16-233 CANCELLED AT 2023-07-11T18:30:33 DUE TO TIME LIMIT ***
srun: Job step aborted: Waiting up to 62 seconds for job step to finish.
srun: error: compute-e-16-233: task 0: Killed
(base) [rh183@login03 ~]$ 
AP /n/data1/hms/neurobio/sabatini/rich/github_repos/NB 

git pull
(base) [rh183@login03 NBAP]$ git pull



## Activate environment

In [67]:
name_env = '/n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR'

send_expect(f'conda activate {name_env}');

in/bashpty -p interactive -t 0-00:30:00 --mem=4G /b 

conda activate /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR

srun: job 12693310 queued and waiting for resources

srun: job 12693310 has been allocated resources

conda activate /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR

(base) [rh183@compute-a-16-161 NBAP]$ 


## Prepare commands

In [68]:
for date in dates_toUse:

    mouse = 'mouse_0322N'
    # date = '20230501'

    name_slurm = mouse + '_' + date[-4:]

    dir_FR_template       = str(Path('/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm') / mouse / 'run_20230701' / '20230430' / 'jobNum_0')
    dir_FR_current        = str(Path('/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm') / mouse / 'run_20230701' /  date / 'jobNum_0')

    dir_saveOutputs = str(Path('/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm') / mouse / 'run_20230701' / date / 'aligned_TCA')


    name_job = 'jobNum_'


    path_script_remote      = str(Path(r'/n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/1_make_consistent_fr_factors.py'))  ## path to a copy of the script (.py) file to run within the repo on the server

    path_dispatcher_local   = str(Path(r'/media/rich/Home_Linux_partition/github_repos/NBAP/behavioral_drift_analysis/').resolve() / 'dispatcher.py')  ## path to the dispatcher.py file on local computer
    path_dispatcher_remote  = (Path(dir_saveOutputs) / 'dispatcher.py').as_posix()  ## path to where you want the dispatcher.py file to be copied onto the server

    ## Expectation for dispatch args: 
    # path_script, dir_save, dir_videos, dir_ROIs, name_job, name_slurm, name_env


    commands = {
    #     'make_dir': f"mkdir -p {dir_data_remote}",
    #     'copy': f"cp -r {dir_s2p_MICROSCOPE} {dir_data_remote}",
    #     'dispatch': f"python {str(path_dispatcher_remote)} {dir_saveOutputs} {path_script_remote} {name_job} {name_slurm} {dir_videos}",  ## dispatcher expecting these args as inputs
    'dispatch': f"\
    python3 \
    {path_dispatcher_remote} \
    {path_script_remote} \
    {dir_saveOutputs} \
    {dir_FR_template} \
    {dir_FR_current} \
    {name_job} \
    {name_slurm} \
    {name_env}",
    }
    display(commands)
    
    sftp.mkdir_p(str(Path(path_dispatcher_remote).parent))
    sftp.sftp.put(str(path_dispatcher_local), str(path_dispatcher_remote));
    
    send_expect(commands['dispatch']);

{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230503/aligned_TCA/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/1_make_consistent_fr_factors.py     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230503/aligned_TCA     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230430/jobNum_0     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230503/jobNum_0     jobNum_     mouse_0322N_0503     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR'}

ini/rich/virtual_envs/FR
AP]$ 


{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230505/aligned_TCA/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/1_make_consistent_fr_factors.py     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230505/aligned_TCA     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230430/jobNum_0     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230505/jobNum_0     jobNum_     mouse_0322N_0505     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR'}

rtual_envs/FRum_     mouse_0322N_0503     /n/data1/hms/neurobio/sabatini/rich/vi 

    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230505/aligned_TCA/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/1_make_consistent_fr_factors.py     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230505/aligned_TCA     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230430/jobNum_0     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230505/jobNum_0     jobNum_     mouse_0322N_0505     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR

dispatcher environment: /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR

Copying /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/1_make_consistent_fr_factors.py to /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N

{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230506/aligned_TCA/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/1_make_consistent_fr_factors.py     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230506/aligned_TCA     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230430/jobNum_0     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230506/jobNum_0     jobNum_     mouse_0322N_0506     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR'}

...

dispatcher environment: /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR

    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230525/aligned_TCA/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/1_make_consistent_fr_factors.py     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230525/aligned_TCA     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230430/jobNum_0     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230525/jobNum_0     jobNum_     mouse_0322N_0525     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR

Copying /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/1_make_consistent_fr_factors.py to /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230524/aligned_TCA/1_make_consistent_fr_factors.py

Submitting job:

{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230526/aligned_TCA/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/1_make_consistent_fr_factors.py     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230526/aligned_TCA     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230430/jobNum_0     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230526/jobNum_0     jobNum_     mouse_0322N_0526     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR'}

rtual_envs/FRum_     mouse_0322N_0525     /n/data1/hms/neurobio/sabatini/rich/vi 
dispatcher environment: /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR

    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230526/aligned_TCA/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/1_make_consistent_fr_factors.py     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230526/aligned_TCA     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230430/jobNum_0     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230526/jobNum_0     jobNum_     mouse_0322N_0526     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR

Copying /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/1_make_consistent_fr_factors.py to /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/

{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230530/aligned_TCA/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/1_make_consistent_fr_factors.py     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230530/aligned_TCA     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230430/jobNum_0     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230530/jobNum_0     jobNum_     mouse_0322N_0530     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR'}

rtual_envs/FRum_     mouse_0322N_0526     /n/data1/hms/neurobio/sabatini/rich/vi 
dispatcher environment: /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR

    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230530/aligned_TCA/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/1_make_consistent_fr_factors.py     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230530/aligned_TCA     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230430/jobNum_0     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230530/jobNum_0     jobNum_     mouse_0322N_0530     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR

Copying /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/1_make_consistent_fr_factors.py to /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/

## Run with serverteeny

In [257]:
run_file = {
    "name": "Rich",
    "o2_acct": "shl143",
    "notes": "testing serverteeny for FR",
    "command": f"sbatch /n/data1/hms/neurobio/sabatini/rich/serverteeny_submitter.sh {commands['dispatch']}",
}

In [258]:
path_temp = str(Path(tempfile.gettempdir()) / 'run.json')
bnpm.file_helpers.json_save(
    obj=run_file,
    filepath=path_temp,
)

In [259]:
sftp.sftp.put(
    localpath=path_temp,
    remotepath=str(Path('/n/data1/hms/neurobio/sabatini/serverteeny/run') / Path(path_temp).name),
);

### check on job

## Clean up

In [ ]:
## delete password
del pw
gc.collect()

## close sftp
if sftp: sftp.close()
if transport: transport.close()
del sftp
del transport
    
## close ssh
ssh_t.close()
del ssh_t
ssh_c.close()
del ssh_c
gc.collect()